# 语义分块
文本分块是检索增强生成 （RAG） 中的一个重要步骤，其中大型文本正文被划分为有意义的段以提高检索准确性。与固定长度分块不同，语义分块根据句子之间的内容相似性来拆分文本。

### 断点方法：
- 百分位数：查找所有相似性差异的第 X 个百分位数，并拆分下降大于此值的块。
- 标准差：当相似性低于平均值 X 个标准差以上时，进行拆分。
- 四分位距 （IQR）：使用四分位距 （Q3 - Q1） 来确定分割点。

此笔记本 使用 percentile 方法 实现语义分块，并评估其在示例文本上的性能。

In [2]:
#导入模块
import fitz
import os
import numpy as np
import json
from openai import OpenAI

# 提取文本内容
为了实现RAG，首先需要准备文本内容

In [3]:
#提取文本内容
def extract_text_from_pdf(pdf_path):
    """从PDF文件中提取文本内容"""
    doc = fitz.open(pdf_path)
    text = ""
    for page in doc:
        text += page.get_text()
    return text.strip()

pdf_path = "data/sample.pdf"
extracted_text = extract_text_from_pdf(pdf_path)
print(extracted_text)


百炼系列平板电脑产品介绍 
踏入数字世界的宽广领域，百炼平板电脑系列以其卓越的设计与顶尖技术，为您的工作与娱乐带来前所未有的体验。 
 
百炼Tab Xplorer P1 —— 探索无界视野： 配备12.9 英寸Liquid Retina XDR 显示屏，分辨率达到2732 x 2048 像素，支持ProMotion 自适应刷新率技术（最高
120Hz），无论是高清视频还是专业绘图，细腻与流畅并存。搭载1TB 超大存储与16GB RAM，配合M1 芯片的强大性能，轻松驾驭复杂应用与大型游戏。
10000mAh 电池确保全天候续航，四扬声器环绕立体声，为您带来影院级视听享受。参考售价：8999 - 9999。 
通义Vivid Tab 7 —— 智能办公新境界： 拥有10.5 英寸2560 x 1600 像素Super AMOLED 显示屏，色彩鲜活，细节丰富。8GB RAM 与256GB 存储空间，结合
高效的处理器，确保多任务处理顺滑无阻。支持S Pen 手写笔，灵感随时记录。7500mAh 电池，配合智能电源管理，满足全天工作需求。内置多重生物识别技
术，包括面部识别与指纹解锁，安全便捷。参考售价：4499 - 4899。 
星尘Tab S9 Pro —— 创意与效率的完美融合： 采用12.4 英寸2800 x 1752 像素超窄边框设计，屏下摄像头技术，最大化显示区域。512GB 存储与12GB 
RAM，搭载高效能处理器，流畅运行各类创作软件。9000mAh 电池与65W 超级快充，快速回血，创意不中断。支持外接键盘与触控笔，变身移动工作站。参考
售价：6999 - 7499。 
百炼Ace Tab Ultra —— 游戏与娱乐的旗舰选择： 配备11 英寸2560 x 1600 像素屏幕，支持HDR10+，色彩表现力惊人。12GB RAM 与512GB 存储空间，专为
高性能游戏优化。7800mAh 电池与液冷散热系统，确保长时间游戏稳定不发热。四扬声器杜比全景声音效，沉浸式游戏体验。参考售价：5999 - 6499。 
百炼Zephyr Tab Z9 —— 轻盈便携的智慧伴侣： 采用轻薄8.4 英寸2560 x 1600 像素OLED 屏幕，携带方便。6GB RAM 与128GB 存储，满足日常娱乐与学习需
求。6000mAh 电池提供持久续航，支持快速充

# 设置OpenAPI Client

In [4]:
client = OpenAI(
    base_url="https://api.siliconflow.cn/v1/", 
    api_key=os.getenv("SILLICONFLOW_API_KEY")
    )

# 创建语句型的Embedding

In [16]:
def get_embedding(text, model="BAAI/bge-large-zh-v1.5"):
    """创建语句型的Embedding

    Args:
        text (_type_): _description_
        model (str, optional): _description_. Defaults to "BAAI/bge-large-zh-v1.5".
    """
    response = client.embeddings.create(
        model=model, 
        input=text)
    return np.array(response.data[0].embedding)
#这里最重要，需要将文本按句号分割成句子
sentences = extracted_text.split("。") # 将文本按句号分割成句子
print(f"句子数量: {len(sentences)}")
embeddings = []
for sentence in sentences:
    if len(sentence) > 0:
        # print(f"句子：{sentence}")
        embedding = get_embedding(sentence)
        embeddings.append(embedding)

print(f"句子嵌入的数量：{len(embeddings)}")

句子数量: 29
句子嵌入的数量：28


In [17]:
def cosine_similarity(vec1, vec2):
    """
    Computes cosine similarity between two vectors.

    Args:
    vec1 (np.ndarray): First vector.
    vec2 (np.ndarray): Second vector.

    Returns:
    float: Cosine similarity.
    """
    return np.dot(vec1, vec2) / (np.linalg.norm(vec1) * np.linalg.norm(vec2))
#计算相似性
similarities = []
for i in range(len(embeddings)-1):
    similarities.append(cosine_similarity(embeddings[i], embeddings[i+1]))


# 实现语义分块
通过不同的技术来实现语义分块

In [126]:
def compute_breakpoints(similarities, method="percentile", threshold=20):
    """计算断点

    Args:
        text (_type_): 文本
        method (str, optional): 百分比. Defaults to "percentile".
        threshold (float, optional): 阈值. Defaults to 90.
    """
    if method == "percentile":
        # 计算百分位数
        threshold_value = np.percentile(similarities, threshold, method="linear")
        
    elif method == "interquartile":
        # 计算四分位距
        q1, q3 = np.percentile(similarities, [25, 75])
        # 计算阈值
        threshold_value = q1-1.5*(q3-q1)
        
    elif method == "standard_deviation":
        # 计算平均值
        mean_similarity = np.mean(similarities)
        # 计算标准差
        std_deviation = np.std(similarities)
        # 计算阈值
        threshold_value = mean_similarity - threshold * std_deviation
    else:
        raise ValueError(f"不支持的方法：{method}")
    
    # 计算断点
    breakpoints = [i for i, sim in enumerate(similarities) if sim < threshold_value]
    return breakpoints
#threshold=20 表示相似度低于20%的句子，作为断点
breakpoints = compute_breakpoints(similarities, 
                                  method="percentile",
                                  threshold=40)
print(f"断点数量：{len(breakpoints)}")
print(f"断点：{breakpoints}")


断点数量：11
断点：[1, 4, 7, 10, 13, 14, 15, 19, 20, 24, 25]


#  根据Breakpoints 分割文本

In [127]:
def split_text_by_breakpoints(sentences, breakpoints):
    """根据Breakpoints 分割文本
    """
    chunks = []
    start_index = 0
    current_chunk = ""
    for bp in breakpoints:
        print(f"start_index：{start_index}，bp：{bp}")
        current_chunk = "。".join(sentences[start_index:bp]) + "。"
        start_index = bp
        if len(current_chunk.strip()) == 0:
            continue
        print(f"current_chunk：\n{current_chunk}")
        
        chunks.append(current_chunk)
        
    # 处理最后一个块
    if start_index < len(sentences):
        current_chunk = "。".join(sentences[start_index:len(sentences)])
        print(f"current_chunk：\n{current_chunk}")
        chunks.append(current_chunk)
    return chunks

chunks = split_text_by_breakpoints(sentences, breakpoints)
print(f"分割后的文本数量：{len(chunks)}")




start_index：0，bp：1
current_chunk：
百炼系列平板电脑产品介绍 
踏入数字世界的宽广领域，百炼平板电脑系列以其卓越的设计与顶尖技术，为您的工作与娱乐带来前所未有的体验。
start_index：1，bp：4
current_chunk：
 
 
百炼Tab Xplorer P1 —— 探索无界视野： 配备12.9 英寸Liquid Retina XDR 显示屏，分辨率达到2732 x 2048 像素，支持ProMotion 自适应刷新率技术（最高
120Hz），无论是高清视频还是专业绘图，细腻与流畅并存。搭载1TB 超大存储与16GB RAM，配合M1 芯片的强大性能，轻松驾驭复杂应用与大型游戏。
10000mAh 电池确保全天候续航，四扬声器环绕立体声，为您带来影院级视听享受。
start_index：4，bp：7
current_chunk：
参考售价：8999 - 9999。 
通义Vivid Tab 7 —— 智能办公新境界： 拥有10.5 英寸2560 x 1600 像素Super AMOLED 显示屏，色彩鲜活，细节丰富。8GB RAM 与256GB 存储空间，结合
高效的处理器，确保多任务处理顺滑无阻。
start_index：7，bp：10
current_chunk：
支持S Pen 手写笔，灵感随时记录。7500mAh 电池，配合智能电源管理，满足全天工作需求。内置多重生物识别技
术，包括面部识别与指纹解锁，安全便捷。
start_index：10，bp：13
current_chunk：
参考售价：4499 - 4899。 
星尘Tab S9 Pro —— 创意与效率的完美融合： 采用12.4 英寸2800 x 1752 像素超窄边框设计，屏下摄像头技术，最大化显示区域。512GB 存储与12GB 
RAM，搭载高效能处理器，流畅运行各类创作软件。
start_index：13，bp：14
current_chunk：
9000mAh 电池与65W 超级快充，快速回血，创意不中断。
start_index：14，bp：15
current_chunk：
支持外接键盘与触控笔，变身移动工作站。
start_index：15，bp：19
current_chunk：
参考
售价：6999 - 7499。 
百炼Ace 

# 创建语义块的Embedding

In [128]:
def create_chunk_embeddings(chunks):
    """创建语义块的Embedding
    """
    embeddings = []
    for chunk in chunks:
        embedding = get_embedding(chunk)
        embeddings.append(embedding)
    return embeddings

chunk_embeddings = create_chunk_embeddings(chunks)
print(f"语义块的Embedding数量：{len(chunk_embeddings)}")
print(f"语义块的Embedding：{chunk_embeddings[0]}")



语义块的Embedding数量：12
语义块的Embedding：[ 0.00026164 -0.01669292 -0.03890266 ... -0.04353886  0.04677643
 -0.03214261]


# 创建语义搜索
用cosine相似度去，查找语义最近的语块

In [129]:
def search_semantic(query, chunk_embeddings, text_chunks, top_k=5):
    """创建语义搜索
    """
    # 创建查询的Embedding
    query_embedding = get_embedding(query)
    # 计算相似性
    similarities = []
    for i, chunk in enumerate(chunk_embeddings):
        similarity = cosine_similarity(query_embedding, chunk)
        similarities.append((similarity, i))
    similarities.sort(key=lambda x: x[0], reverse=True)
    top_results = similarities[:top_k]
    return [text_chunks[i] for _, i in top_results]

# 测试语义搜索
query = "参考售价4000元-5000元"
semantic_chunks = search_semantic(query, chunk_embeddings, chunks, top_k=2)
print(f"语义搜索结果：\n{semantic_chunks}")



语义搜索结果：
['参考售价：4499 - 4899。 \n星尘Tab S9 Pro —— 创意与效率的完美融合： 采用12.4 英寸2800 x 1752 像素超窄边框设计，屏下摄像头技术，最大化显示区域。512GB 存储与12GB \nRAM，搭载高效能处理器，流畅运行各类创作软件。', '参考售价：8999 - 9999。 \n通义Vivid Tab 7 —— 智能办公新境界： 拥有10.5 英寸2560 x 1600 像素Super AMOLED 显示屏，色彩鲜活，细节丰富。8GB RAM 与256GB 存储空间，结合\n高效的处理器，确保多任务处理顺滑无阻。']


In [130]:
import random


with open("data/sample.json", "r") as f:
    json_data = json.load(f)
index = random.randint(0, len(json_data)-1)
query = json_data[index]["问题"]
top_results = search_semantic(query, chunk_embeddings, chunks, top_k=3)
print(f"查询：{query}")

for i, result in enumerate(top_results):
    print(f"Context{i+1}：\n{result}")
    print("-"*100)


查询：低于1000元的平板电脑有哪些？
Context1：
参考售价：3499 - 3799。 
 
每一款百炼平板电脑都是对极致体验的追求，旨在成为您探索数字世界的理想伙伴。选择百炼，开启您的智能生活新篇章。
----------------------------------------------------------------------------------------------------
Context2：
参考售价：4499 - 4899。 
星尘Tab S9 Pro —— 创意与效率的完美融合： 采用12.4 英寸2800 x 1752 像素超窄边框设计，屏下摄像头技术，最大化显示区域。512GB 存储与12GB 
RAM，搭载高效能处理器，流畅运行各类创作软件。
----------------------------------------------------------------------------------------------------
Context3：
参考售价：5999 - 6499。 
百炼Zephyr Tab Z9 —— 轻盈便携的智慧伴侣： 采用轻薄8.4 英寸2560 x 1600 像素OLED 屏幕，携带方便。6GB RAM 与128GB 存储，满足日常娱乐与学习需
求。6000mAh 电池提供持久续航，支持快速充电。
----------------------------------------------------------------------------------------------------


# 基于搜索结果生成回复

In [131]:
system_prompt = """
你是一个专业的销售顾问，擅长根据用户的问题，根据给定的上下文，给出最严格的回答。
如果上下文没有提供相关信息，请回答“没有找到相关信息”。
"""
def generate_response(system_prompt, user_prompt, model="Qwen/Qwen2-7B-Instruct"):
    """基于搜索结果生成回复
    """
    # 创建OpenAI Client
    response = client.chat.completions.create(
        model= model,
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": user_prompt}
        ]
    )
    return response

user_prompt = "\n".join([f"上下文{i+1}：\n{result}" for i, result in enumerate(top_results)])
user_prompt = f"""{user_prompt} \n 问题：{query}"""
response = generate_response(system_prompt, user_prompt)
print(f"回复：{response.choices[0].message.content}")
print(f"理想回答：{json_data[index]['理想回答']}")


回复：没有找到相关信息
理想回答：没有找到相关信息


In [132]:
evaluation_system_prompt = """
你是一个专业的销售顾问，擅长根据用户的问题，根据给定的上下文，给出最严格的回答。
如果销售顾问的回答与”理想回答“完全一致，给出1分， 完全不同，给出0分，部分相同，给出0.5分。
"""

evaluation_user_prompt = f"""
问题：{query}
理想回答：{json_data[index]["理想回答"]}
销售顾问的回答：{response.choices[0].message.content}
"""

evaluation_response = generate_response(evaluation_system_prompt, evaluation_user_prompt)
print(f"评估：{evaluation_response.choices[0].message.content}")

评估：回答结果：1分
